In [16]:
import json
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# set the filepath and load
data_path = "../DC5-Data/"
fp = data_path + "Sensor Data and Maps/StHimarkNeighborhoodShapefiles/StHimark.shp"
#reading the file stored in variable fp
map_df = gpd.read_file(fp)

In [4]:
map_df = map_df.to_crs(epsg=4326)

In [5]:
data = map_df.to_json()
data = json.loads(data)

In [6]:
neighborhood_df = pd.DataFrame({'neighborhood': [x['properties']['Nbrhood'] for x in data['features']]})

In [7]:
sensor_locations = pd.read_csv(data_path + "Sensor Data and Maps/StaticSensorLocations.csv")
sensor_locations_gdf = gpd.GeoDataFrame(
    sensor_locations, geometry=gpd.points_from_xy(sensor_locations.Long, sensor_locations.Lat))
sensor_locations_gdf

,Sensor-id,Lat,Long,geometry
0,12,0.20764,-119.81556,POINT (-119.81556 0.20764)
1,15,0.16849,-119.79033,POINT (-119.79033 0.16849)
2,13,0.15979,-119.80715,POINT (-119.80715 0.15979)
3,11,0.04147,-119.82861,POINT (-119.82861 0.04147)
4,6,0.12180,-119.90430,POINT (-119.90430 0.12180)
5,1,0.15689,-119.95940,POINT (-119.95940 0.15689)
6,9,0.18299,-119.85558,POINT (-119.85558 0.18299)
7,14,0.12180,-119.79265,POINT (-119.79265 0.12180)
8,4,0.15109,-119.90952,POINT (-119.90952 0.15109)


In [8]:
hospitals = pd.read_csv('../DC5-data/hospitals.csv')

In [57]:
def discretize_scale(color_arr):
    new_scale = []
    n = len(color_arr)
    h = 1.0/(n)
    c_order = h * np.arange(n+1)
    for i in range(n):
        new_scale.append([c_order[i], color_arr[i]])
        # To have clear boundaries between colors in the colorbar
        if i < (n):
            new_scale.append([c_order[i+1], color_arr[i]])

    return new_scale

In [66]:
px.colors.qualitative.Light24[-n_neighborhoods:]

['#0DF9FF',
 '#F6F926',
 '#FF9616',
 '#479B55',
 '#EEA6FB',
 '#DC587D',
 '#D626FF',
 '#6E899C',
 '#00B5F7',
 '#B68E00',
 '#C9FBE5',
 '#FF0092',
 '#22FFA7',
 '#E3EE9E',
 '#86CE00',
 '#BC7196',
 '#7E7DCD',
 '#FC6955',
 '#E48F72']

In [76]:
minx, miny, maxx, maxy = map_df.geometry.total_bounds
n_neighborhoods = len(neighborhood_df)
fig = go.Figure()
fig.add_trace(
    go.Choroplethmapbox(geojson=data,
                                  colorscale=discretize_scale(px.colors.qualitative.Light24[-n_neighborhoods:]),
                                  colorbar=dict(ticktext=neighborhood_df["neighborhood"].unique(),
                                                  tickvals=[n_neighborhoods*(i/n_neighborhoods) for i in range(n_neighborhoods)],
                                                  tickmode="array"),
                           locations=neighborhood_df["neighborhood"], z=neighborhood_df.index, 
                                  featureidkey="properties.Nbrhood"
                           )
)

fig.add_trace(
    go.Scattermapbox(lat=hospitals["lat"],
                        lon=hospitals["long"],
                              marker=dict(color="white",
                                         size=10),
                    text="Hospital",
                    showlegend=False)
)

fig.add_trace(
    go.Scattermapbox(lat=sensor_locations["Lat"],
                        lon=sensor_locations["Long"],
                              marker=go.scattermapbox.Marker(color="red"),
                    text=sensor_locations['Sensor-id'],

                    showlegend=False)
)

fig.add_trace(
    go.Scattermapbox(lat=[0.162679],
                        lon=[-119.784825],
                              marker=dict(color="yellow", size=10),
                    text="Nuclear Plant",
                    showlegend=False)
)
fig.update_layout(mapbox_style="carto-positron",
                 title_text='St. Himark Landmarks',
                 legend_title='Neighborhoods',
                  mapbox_zoom=10, mapbox_center={"lon": (minx+maxx)/2, "lat": (miny+maxy)/2})
                  
fig.show()

In [46]:
[100*(i/n_neighborhoods + 1/(2*n_neighborhoods)) for i in range(n_neighborhoods)]

[2.631578947368421,
 7.894736842105263,
 13.157894736842104,
 18.421052631578945,
 23.684210526315788,
 28.947368421052634,
 34.210526315789465,
 39.473684210526315,
 44.73684210526315,
 50.0,
 55.263157894736835,
 60.526315789473685,
 65.78947368421052,
 71.05263157894737,
 76.3157894736842,
 81.57894736842105,
 86.84210526315789,
 92.10526315789474,
 97.36842105263158]

In [48]:
neighborhood_df["neighborhood"].unique()

array(['Palace Hills', 'Northwest', 'Old Town', 'Safe Town', 'Southwest',
       'Downtown', 'Wilson Forest', 'Scenic Vista', 'Broadview',
       'Chapparal', 'Terrapin Springs', 'Pepper Mill', 'Cheddarford',
       'Easton', 'Weston', 'Southton', 'Oak Willow', 'East Parton',
       'West Parton'], dtype=object)